In [1]:
import os
import paddle
from paddle.io import Dataset
import numpy as np
import matplotlib.pyplot as plt
paddle.set_default_dtype("float64")

In [2]:
from Picture import *

In [3]:
#参数
label_dict={"0":"汽车","1":"摩托车","2":"货车"}
data_src="data/"
train_folder="work/train/"
eval_folder="work/eval/"
batch=10
epoch=10
learning_rate=0.001
show_step=10

In [4]:
class dataset(Dataset):
    def __init__(self,mode="train"):
        super().__init__()
        if mode=="train":
            self.folder=train_folder
        else:
            self.folder=eval_folder
        self.dir=os.listdir(self.folder)
        self.len=len(self.dir)
    def __getitem__(self,index):
        p=Picture.load(os.path.join(self.folder,self.dir[index]))
        return p.data,p.label
    def __len__(self):
        return self.len

In [5]:
train_loader=paddle.io.DataLoader(dataset("train"),batch_size=batch,shuffle=True)
eval_loader=paddle.io.DataLoader(dataset("eval"),batch_size=batch,shuffle=True)

In [6]:
for _,data in enumerate(train_loader()):
    print(data[0])
    break;

Tensor(shape=[10, 43200], dtype=float64, place=Place(cpu), stop_gradient=True,
       [[0.38431373, 0.49803922, 0.46274510, ..., 0.84705882, 0.85490196,
         0.89411765],
        [0.21176471, 0.21176471, 0.21176471, ..., 0.29803922, 0.29803922,
         0.29803922],
        [0.54117647, 0.57647059, 0.56470588, ..., 0.73333333, 0.72941176,
         0.73725490],
        ...,
        [0.89019608, 0.89019608, 0.91372549, ..., 0.93725490, 0.94509804,
         0.94901961],
        [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         1.        ],
        [0.92156863, 0.77647059, 0.64313725, ..., 0.62352941, 0.61960784,
         0.60392157]])


In [7]:
class MyDNN(paddle.nn.Layer):  
    def __init__(self):  
        super(MyDNN,self).__init__()  
        self.li = paddle.nn.LayerList([  
            paddle.nn.Linear(in_features=3*120*120, out_features=4096),  
            paddle.nn.Linear(in_features=4096, out_features=2048),  
            paddle.nn.Linear(in_features=2048, out_features=1024),  
            paddle.nn.Linear(in_features=1024, out_features=512),  
            paddle.nn.Linear(in_features=512, out_features=3)  
        ])  
        self.ac = paddle.nn.LayerList([  
            paddle.nn.ReLU(),  
            paddle.nn.ReLU(),  
            paddle.nn.ReLU(),  
            paddle.nn.ReLU()  
        ])
  
    def forward(self, x):  
        for i in range(len(self.li) - 1):
            x = self.li[i](x)  
            x = self.ac[i](x)  
        x = self.li[-1](x)  
        return x

In [8]:
model=MyDNN()
model.train()
ce=paddle.nn.CrossEntropyLoss(soft_label=True)
op=paddle.optimizer.Adam(learning_rate=learning_rate,parameters=model.parameters())

for i in range(epoch):
    j=0
    for data in train_loader():
        pred=model(data[0])
        loss=ce(pred,data[1])
        loss.backward()
        op.step();
        op.clear_grad()
        j+=1
        if (j%show_step==0):
            print(f"Epoch {i}, Step {j}, Loss: {loss.numpy()}")  

Epoch 0, Step 10, Loss: 19.735817711823415
Epoch 0, Step 20, Loss: 4.006562488610657
Epoch 0, Step 30, Loss: 2.214770564584355
Epoch 0, Step 40, Loss: 1.0256445946836705
Epoch 0, Step 50, Loss: 1.2066299200597193
Epoch 0, Step 60, Loss: 1.0780569898296275
Epoch 0, Step 70, Loss: 0.9495394673653053
Epoch 0, Step 80, Loss: 0.9308782343600835
Epoch 0, Step 90, Loss: 0.8039837502959133
Epoch 0, Step 100, Loss: 0.9268718462402781
Epoch 0, Step 110, Loss: 1.3599432680980441
Epoch 0, Step 120, Loss: 0.8261654023292955
Epoch 0, Step 130, Loss: 1.6232411621243297
Epoch 0, Step 140, Loss: 0.825197209720886
Epoch 1, Step 10, Loss: 0.9868472220161466
Epoch 1, Step 20, Loss: 0.564004343473581
Epoch 1, Step 30, Loss: 0.9105506785871651
Epoch 1, Step 40, Loss: 0.5053959009182681
Epoch 1, Step 50, Loss: 0.7273342022569811
Epoch 1, Step 60, Loss: 1.1478279569645222
Epoch 1, Step 70, Loss: 1.117349371548679
Epoch 1, Step 80, Loss: 0.980481673537876
Epoch 1, Step 90, Loss: 0.683848902893583
Epoch 1, Step

In [9]:
paddle.save(model.state_dict(),"a")